In [10]:
import pandas as pd
import numpy as np
import string

In [14]:
def preprocess_word(word):
    word = word.lower()
    word = ''.join([c for c in word if c in string.ascii_lowercase])
    return word

def preprocess(s):
    s = [preprocess_word(w) for w in s.split(' ')]
    return [w for w in s if w]

In [15]:
paths = 'financial_advice', 'legal_advice', 'medical_advice', 'general'
data = {}
for path in paths:
    with open(f"red_question_data/{path}.txt") as f:
        data[path] = [preprocess(x.strip()) for x in f.readlines()]

In [7]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [5]:
from collections import Counter

In [41]:
all_dfs = []
for key in data:
    count_col = f'{key}'
    df = pd.DataFrame(
        Counter(flatten(data[key])).most_common()).rename(
            columns={0: 'word', 1: count_col}
    )
    df[count_col + '_freq'] = df[count_col] / df[count_col].sum()
    
    all_dfs.append(df)

In [42]:
from functools import reduce

In [43]:
df_merged = reduce(lambda left,right: pd.merge(left, right, on=['word'],
                                            how='outer'), all_dfs).fillna(0)

In [59]:
df_merged = df_merged.set_index('word')

In [83]:
!mkdir words

In [87]:
for key in ['financial_advice', 'legal_advice', 'medical_advice']:
    df_merged[f'{key}_excess'] = np.log(df_merged[f'{key}_freq']) - np.log(df_merged.general_freq + 1e-10)
    bad_words = df_merged[(df_merged[f'{key}_excess'] > 4) & (df_merged[key] > 10)]
    bad_words = bad_words[bad_words.index.str.len() > 4]
    with open(f'words/{key}.txt', 'w') as f:
        f.write('\n'.join(sorted(bad_words.index)))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [76]:
financial_words = df_merged[(df_merged['financial_excess'] > 3.5) & (df_merged['financial_advice'] > 10)]
financial_words

,financial_advice,financial_advice_freq,legal_advice,legal_advice_freq,medical_advice,medical_advice_freq,general,general_freq,financial_excess
word,,,,,,,,,
credit,9052.0,0.013955,458.0,0.000574,0.0,0.000000,647.0,2.264701e-04,4.121010
k,5507.0,0.008490,237.0,0.000297,14.0,0.000026,338.0,1.183105e-04,4.273344
loan,3533.0,0.005447,249.0,0.000312,0.0,0.000000,173.0,6.055538e-05,4.499225
debt,3057.0,0.004713,448.0,0.000561,2.0,0.000004,251.0,8.785781e-05,3.982351
ira,2775.0,0.004278,12.0,0.000015,0.0,0.000000,22.0,7.700684e-06,6.319966
...,...,...,...,...,...,...,...,...,...
looming,11.0,0.000017,1.0,0.000001,0.0,0.000000,1.0,3.500311e-07,3.880225
amortization,11.0,0.000017,0.0,0.000000,0.0,0.000000,0.0,0.000000e+00,12.041118
magi,11.0,0.000017,0.0,0.000000,0.0,0.000000,0.0,0.000000e+00,12.041118


In [78]:
list(financial_words.index)

['credit',
 'k',
 'loan',
 'debt',
 'ira',
 'loans',
 'roth',
 'savings',
 'taxes',
 'income',
 'score',
 'fund',
 'invest',
 'retirement',
 'mortgage',
 'funds',
 'payment',
 'finance',
 'ca',
 'investment',
 'vanguard',
 'balance',
 'payments',
 'finances',
 'traditional',
 'cc',
 'refinance',
 'contributions',
 'expenses',
 'collections',
 'investments',
 'mutual',
 'rewards',
 'index',
 'contribution',
 'hsa',
 'xpost',
 'pf',
 'bonds',
 'filing',
 'rollover',
 'debts',
 'secured',
 'mint',
 'budgeting',
 'consolidate',
 'taxable',
 'apr',
 'consolidation',
 'pension',
 'equity',
 'au',
 'refinancing',
 'inheritance',
 'advisor',
 'contributing',
 'expense',
 'brokerage',
 'financing',
 'mintcom',
 'allocation',
 'rpersonalfinance',
 'repayment',
 'withdraw',
 'amex',
 'fidelity',
 'dependent',
 'payoff',
 'iras',
 'deduction',
 'lump',
 'maxing',
 'gains',
 'maxed',
 'tsp',
 'utilization',
 'deductions',
 'ally',
 'withholding',
 'sallie',
 'mae',
 'fico',
 'ynab',
 'deductible',


In [68]:
financial_df.iloc[np.arange(0, len(financial_df), len(financial_df)//40)][['financial_advice', 'general', 'financial_excess']]

,financial_advice,general,financial_excess
word,,,
pf,264.0,0.0,15.219172
cfp,18.0,0.0,12.533594
lendingclub,11.0,0.0,12.041118
refinancing,155.0,6.0,4.734233
deferment,34.0,2.0,4.315686
allocating,12.0,1.0,3.967236
lenders,35.0,4.0,3.651598
priorities,32.0,5.0,3.338856
meantime,15.0,3.0,3.091958
